In [1]:
import pandas as pd
import glob
import numpy as np
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score

# Sample 30 from each OpenAI Label

In [18]:
TestDF=pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/Sample_1326_For_Testing.json", orient='records')

In [19]:
TestDF.head(1)

,date_unixtime,reply,tfSent,VADERSent,AfinnSent,TextBlobSent,subreddit,num_words,openAI-classification
0,1671323910,Buy silver and xrp just a opinion. Been diggin...,negative,negative,negative,negative,absoluteTruth1776,88,0


In [20]:
TestDF['openAI-classification'].value_counts()

0    719
2    363
3    119
5     52
4     38
1     35
Name: openAI-classification, dtype: int64

In [21]:
df = TestDF.copy()

In [22]:
sampled_df = pd.concat([
    df[df['openAI-classification'] == i].sample(n=min(len(df[df['openAI-classification'] == i]), 30)) for i in range(0, 6)
])

In [23]:
sampled_df.shape

(180, 9)

In [24]:
sampled_df = sampled_df.sample(frac=1)

In [25]:
sampled_df.head(10)

,date_unixtime,reply,tfSent,VADERSent,AfinnSent,TextBlobSent,subreddit,num_words,openAI-classification
959,1667345713,"Yeah, that person has been here under a name o...",negative,negative,negative,negative,DonaldTrumpJr,58,1
806,1650271665,The world is depopulating already . People dyi...,negative,negative,negative,negative,TrumpSupportersChannel,89,5
751,1640801441,time the floods started to flush out all dead ...,negative,negative,negative,negative,TrumpSupportersChannel,177,5
517,1668486576,Further more on the Clot Shot! If Trump had ev...,negative,negative,negative,negative,RTM,86,4
1306,1673065090,Let’s acknowledge the day the uniparty coordi...,negative,negative,negative,negative,WeTheMedia,75,1
714,1678553997,No more secrets from WE THE PEOPLE THIS IS TH...,negative,negative,negative,negative,RTM,56,5
554,1670445046,The Parents have been lied to and misinformed....,negative,negative,negative,negative,RTM,87,2
323,1658332253,"If this is true, him and all his cronnies shou...",negative,negative,negative,negative,ThePatriotVoice,75,4
136,1657811265,This Jackal Is An Idiot and needs to be remove...,negative,negative,negative,negative,TheTrumpRepublicans,69,4
74,1634937564,Shame on any person that wants to hurt God’s l...,negative,negative,negative,negative,TheTrumpRepublicans,68,4


In [26]:
sampled_df['openAI-classification'] = sampled_df['openAI-classification'].replace([6], 0)

In [27]:
sampled_df.to_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotation.csv")

# Predictions on 1136 test set sample

In [126]:
test_data = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/Sample_1326_For_Testing.json", orient='records')

In [127]:
test_data = test_data.sort_values(by=['date_unixtime'], ascending=False)

In [128]:
test_data = test_data.reset_index(drop=True)

In [129]:
test_data.head(1)

,date_unixtime,reply,tfSent,VADERSent,AfinnSent,TextBlobSent,subreddit,num_words,openAI-classification
0,1680634712,"I keep asking, how is it a local lowly NY DA f...",negative,negative,negative,negative,DonaldTrumpJr,64,1


In [130]:
test_data.tail(1)

,date_unixtime,reply,tfSent,VADERSent,AfinnSent,TextBlobSent,subreddit,num_words,openAI-classification
1325,1611588698,Trump has surrounded himself and has chosen so...,negative,negative,negative,negative,InfoWars.com,50,0


In [131]:
test_data.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/Sample_1326_For_Testing.json", orient='records')

## fastText predictions

In [132]:
fTDF = pd.read_json('/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/fastText/test_data_yTrue_yPred.json', orient='records')

In [133]:
fTDF = fTDF.rename(columns={"subreddit": "channel", "article": "reply"})
fTDF = fTDF.drop(columns=['tfSent', 'VADERSent', 'AfinnSent', 'TextBlobSent'])

In [134]:
fTDF.head(1)

,date_unixtime,reply,channel,num_words,label,y_pred
0,1671323910,Buy silver and xrp just a opinion. Been diggin...,absoluteTruth1776,88,0,2


In [135]:
fTDF.shape

(1326, 6)

## LSTM predictions

In [136]:
lstmDF = pd.read_json('/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/LSTM/test_data_yTrue_yPred.json', orient='records')

In [137]:
lstmDF = lstmDF.rename(columns={"subreddit": "channel", "article": "reply"})
lstmDF = lstmDF.drop(columns=['tfSent', 'VADERSent', 'AfinnSent', 'TextBlobSent'])

In [138]:
lstmDF.head(1)

,date_unixtime,reply,channel,num_words,label,y_pred
0,1680634712,"I keep asking, how is it a local lowly NY DA f...",DonaldTrumpJr,64,1,3


In [139]:
lstmDF.shape

(1326, 6)

## GPT2 predictions

In [140]:
GPT2DF = pd.read_json('/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/OpenAIGPT2/test_data_yTrue_yPred.json', orient='records')

In [141]:
GPT2DF = GPT2DF.rename(columns={"subreddit": "channel", "text": "reply"})
GPT2DF = GPT2DF.drop(columns=['tfSent', 'VADERSent', 'AfinnSent', 'TextBlobSent'])

In [142]:
GPT2DF.head(1)

,date_unixtime,reply,channel,num_words,label,y_pred
0,1680634712,"I keep asking, how is it a local lowly NY DA f...",DonaldTrumpJr,64,1,1


In [143]:
GPT2DF.shape

(1326, 6)

## SVM predictions

In [144]:
SVMDF = pd.read_json('/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/SVM/test_data_yTrue_yPred.json', orient='records')

In [145]:
SVMDF = SVMDF.rename(columns={"subreddit": "channel", "article": "reply"})
SVMDF = SVMDF.drop(columns=['tfSent', 'VADERSent', 'AfinnSent', 'TextBlobSent'])

In [146]:
SVMDF.head(1)

,date_unixtime,reply,channel,num_words,label,y_pred
0,1671323910,Buy silver and xrp just a opinion. Been diggin...,absoluteTruth1776,88,0,0


In [147]:
SVMDF.shape

(1326, 6)

# 180 samples

In [148]:
samples180 = pd.read_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotation.csv")

In [149]:
samples180 = samples180.rename(columns={"subreddit": "channel"})
samples180 = samples180.drop(columns=['tfSent', 'VADERSent', 'AfinnSent', 'TextBlobSent', 'Unnamed: 0'])

In [150]:
samples180.head(1)

,date_unixtime,reply,channel,num_words,openAI-classification
0,1667345713,"Yeah, that person has been here under a name o...",DonaldTrumpJr,58,1


In [151]:
samples180.shape

(180, 5)

In [152]:
# fastText
# Create a new column in LizTest174 to store the label values
samples180['fastText'] = None

# Iterate over each row in LizTest174
for index, row in samples180.iterrows():
    time = row['date_unixtime']
    
    # Check if the reply is in G5df['reply']
    matching_row = fTDF[fTDF['date_unixtime'] == time]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['y_pred']
        samples180.at[index, 'fastText'] = label

In [153]:
# LSTM
# Create a new column in LizTest174 to store the label values
samples180['LSTM'] = None

# Iterate over each row in LizTest174
for index, row in samples180.iterrows():
    time = row['date_unixtime']
    
    # Check if the reply is in G5df['reply']
    matching_row = lstmDF[lstmDF['date_unixtime'] == time]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['y_pred']
        samples180.at[index, 'LSTM'] = label

In [154]:
# GPT2
# Create a new column in LizTest174 to store the label values
samples180['GPT2'] = None

# Iterate over each row in LizTest174
for index, row in samples180.iterrows():
    time = row['date_unixtime']
    
    # Check if the reply is in G5df['reply']
    matching_row = GPT2DF[GPT2DF['date_unixtime'] == time]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['y_pred']
        samples180.at[index, 'GPT2'] = label

In [155]:
# SVM
# Create a new column in LizTest174 to store the label values
samples180['SVM'] = None

# Iterate over each row in LizTest174
for index, row in samples180.iterrows():
    time = row['date_unixtime']
    
    # Check if the reply is in G5df['reply']
    matching_row = SVMDF[SVMDF['date_unixtime'] == time]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['y_pred']
        samples180.at[index, 'SVM'] = label

In [156]:
samples180.shape

(180, 9)

In [157]:
samples180.head(15)

,date_unixtime,reply,channel,num_words,openAI-classification,fastText,LSTM,GPT2,SVM
0,1667345713,"Yeah, that person has been here under a name o...",DonaldTrumpJr,58,1,0,0,0,0
1,1650271665,The world is depopulating already . People dyi...,TrumpSupportersChannel,89,5,4,0,5,2
2,1640801441,time the floods started to flush out all dead ...,TrumpSupportersChannel,177,5,2,3,2,2
3,1668486576,Further more on the Clot Shot! If Trump had ev...,RTM,86,4,3,0,2,0
4,1673065090,Let’s acknowledge the day the uniparty coordi...,WeTheMedia,75,1,1,0,0,1
5,1678553997,No more secrets from WE THE PEOPLE THIS IS TH...,RTM,56,5,3,0,0,2
6,1670445046,The Parents have been lied to and misinformed....,RTM,87,2,2,2,2,2
7,1658332253,"If this is true, him and all his cronnies shou...",ThePatriotVoice,75,4,5,4,5,5
8,1657811265,This Jackal Is An Idiot and needs to be remove...,TheTrumpRepublicans,69,4,4,0,4,4
9,1634937564,Shame on any person that wants to hurt God’s l...,TheTrumpRepublicans,68,4,2,2,0,2


In [174]:
samples180['LSTM'].value_counts()

0    92
2    52
3    17
5     8
4     7
1     4
Name: LSTM, dtype: int64

In [175]:
samples180.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictions.json", orient='records')

# Annotation by Lizzy on 180

In [188]:
Liz180=pd.read_excel("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotation_Lizzy_July18.xlsx")

In [189]:
Liz180 = Liz180.sort_values(by=['date_unixtime'], ascending=False)
Liz180 = Liz180.reset_index(drop=True)
print(Liz180.head(1))
print(Liz180.tail(1))
print(Liz180['Label'].value_counts())

   Unnamed: 0  date_unixtime  \
0         992     1680634712   

                                               reply  Label  L1  L2  L3  
0  I keep asking, how is it a local lowly NY DA f...      1 NaN NaN NaN  
     Unnamed: 0  date_unixtime  \
179          25     1612468113   

                                                 reply  Label  L1  L2  L3  
179  She was promoting a very radical leftist in In...      6 NaN NaN NaN  
6    68
3    27
5    27
4    27
1    18
2    13
Name: Label, dtype: int64


In [193]:
samples180 = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictions.json", orient='records')

In [194]:
samples180 = samples180.sort_values(by=['date_unixtime'], ascending=False)
samples180 = samples180.reset_index(drop=True)
print(samples180.head(1))
print(samples180.tail(1))
print(samples180['SVM'].value_counts())

   date_unixtime                                              reply  \
0     1680634712  I keep asking, how is it a local lowly NY DA f...   

         channel  num_words  openAI-classification  fastText  LSTM  GPT2  SVM  
0  DonaldTrumpJr         64                      1         3     3     1    1  
     date_unixtime                                              reply  \
179     1612468113  She was promoting a very radical leftist in In...   

          channel  num_words  openAI-classification  fastText  LSTM  GPT2  SVM  
179  InfoWars.com         58                      0         0     1     2    0  
0    74
2    46
3    30
5    13
4    10
1     7
Name: SVM, dtype: int64


In [196]:
# MAtch
# Create a new column in LizTest174 to store the label values
samples180['Expert'] = None

# Iterate over each row in LizTest174
for index, row in samples180.iterrows():
    time = row['date_unixtime']
    
    # Check if the reply is in G5df['reply']
    matching_row = Liz180[Liz180['date_unixtime'] == time]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['Label']
        samples180.at[index, 'Expert'] = label

In [198]:
print(samples180.head(1))
print(samples180.tail(1))
print(samples180['Expert'].value_counts())

   date_unixtime                                              reply  \
0     1680634712  I keep asking, how is it a local lowly NY DA f...   

         channel  num_words  openAI-classification  fastText  LSTM  GPT2  SVM  \
0  DonaldTrumpJr         64                      1         3     3     1    1   

  Expert  
0      1  
     date_unixtime                                              reply  \
179     1612468113  She was promoting a very radical leftist in In...   

          channel  num_words  openAI-classification  fastText  LSTM  GPT2  \
179  InfoWars.com         58                      0         0     1     2   

     SVM Expert  
179    0      6  
6    68
3    27
5    27
4    27
1    18
2    13
Name: Expert, dtype: int64


In [201]:
samples180['Expert'] = samples180['Expert'].replace([6], 0)

In [202]:
samples180.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert.json", orient='records')

## Custom Weighted Kappa

In [203]:
samples180 = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert.json", orient='records')

In [204]:
# Define the weight matrix
weightsLR = np.array([
    [0, 0.2, 0.2, 0.3, 0.5, 0.5],
    [0.2, 0, 0.1, 0.2, 0.4, 0.4],
    [0.2, 0.1, 0, 0.2, 0.4, 0.4],
    [0.3, 0.2, 0.2, 0, 0.2, 0.1],
    [0.5, 0.4, 0.4, 0.2, 0, 0.3],
    [0.5, 0.4, 0.4, 0.2, 0.3, 0]
])

def customWeightedKappa(y1, y2, weights):
    # Calculate the confusion matrix
    confusion = confusion_matrix(y1, y2)

    # Normalize the confusion matrix to represent probabilities
    total_ratings = np.sum(confusion)
    confusion = confusion / total_ratings

    # Compute the expected probability matrix under the assumption of independence
    ratings_probs = np.sum(confusion, axis=1)
    expected = np.outer(ratings_probs, ratings_probs)

    # Compute the weighted Kappa score
    kappa = 1 - np.sum(weights * confusion) / np.sum(weights * expected)
    
    return kappa

In [207]:
print(customWeightedKappa(samples180['Expert'], samples180['LSTM'], weights=weightsLR))

0.2591698550444763


In [210]:
print(customWeightedKappa(samples180['Expert'], samples180['SVM'], weights=weightsLR))

0.3265180500404329


In [205]:
print(customWeightedKappa(samples180['Expert'], samples180['fastText'], weights=weightsLR))

0.36127840874802364


In [208]:
print(customWeightedKappa(samples180['Expert'], samples180['GPT2'], weights=weightsLR))

0.4329716485824291


In [211]:
print(customWeightedKappa(samples180['openAI-classification'], samples180['Expert'], weights=weightsLR))

0.5146067415730338


# Choose examples

In [212]:
samples180 = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert.json", orient='records')

In [213]:
samples180.head(1)

,date_unixtime,reply,channel,num_words,openAI-classification,fastText,LSTM,GPT2,SVM,Expert
0,1680634712,"I keep asking, how is it a local lowly NY DA f...",DonaldTrumpJr,64,1,3,3,1,1,1


In [231]:
df = samples180.copy()

In [232]:
mismatch = df[(df['openAI-classification'] != df['GPT2']) & (df['GPT2'] != df['Expert']) & (df['Expert'] != df['openAI-classification'])]

In [233]:
mismatch.shape

(35, 10)

In [234]:
mismatch = mismatch.sort_values(by=['num_words'], ascending=True)
mismatch = mismatch.reset_index(drop=True)

In [235]:
mismatch.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert_mismatch35.json", orient='records')

In [236]:
agreed = df[(df['openAI-classification'] == df['fastText']) & 
            (df['fastText'] == df['LSTM']) &
            (df['LSTM'] == df['GPT2']) &
            (df['GPT2'] == df['SVM']) &
            (df['SVM'] == df['Expert'])]

In [237]:
agreed = agreed.sort_values(by=['num_words'], ascending=True)
agreed = agreed.reset_index(drop=True)

In [238]:
agreed.shape

(16, 10)

In [239]:
agreed.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert_agreed4.json", orient='records')

In [240]:
alldisagree = df[(df['openAI-classification'] != df['fastText']) & 
            (df['fastText'] != df['LSTM']) &
            (df['LSTM'] != df['GPT2']) &
            (df['GPT2'] != df['SVM']) &
            (df['SVM'] != df['Expert'])]

In [242]:
alldisagree = alldisagree.sort_values(by=['num_words'], ascending=True)
alldisagree = alldisagree.reset_index(drop=True)

In [243]:
alldisagree.shape

(23, 10)

In [244]:
alldisagree.to_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples180BasedonOpenAIForAnnotationwithPredictionsWithExpert_alldisagree23.json", orient='records')